In [ ]:
# Задание 3 из 6
# Сохранить взаимодействие для пользователя user_id с объектом item_id:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

# Получить список последних событий для пользователя user_id:
import requests

events_store_url = "http://127.0.0.1:8020"

headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
params = {"user_id": 1337055}

resp = requests.post(events_store_url + "/get", headers=headers, params=params)
if resp.status_code == 200:
    result = resp.json()
else:
    result = None
    print(f"status code: {resp.status_code}")
    
print(result)

# Проверьте, что для пользователя 1127794 в Event Store нет никаких событий. 
# Затем сохраните для этого пользователя последовательно четыре события с объектами: 18734992, 18734992, 7785, 4731479. 
# После чего получите для того же пользователя последние три события:
resp = requests.post(events_store_url + "/get", 
                     headers=headers, 
                     params={"user_id": user_id, "k": 3})
print(resp.json())
# Должно быть {'events': [4731479, 7785, 18734992]}

In [ ]:
# Задание 4 из 6
# Протестируйте работу нового метода. Вы получите онлайн-рекомендации (длиной 3) для пользователя 1291248:

params = {"user_id": 1291248, 'k': 3}
recommendations_url = "http://127.0.0.1:8020"
resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs) 
# Список пустой. Это ожидаемо, так как для пользователя в Event Store пока нет никаких событий, чтобы по ним получить онлайн-рекомендации.

# Добавим событие:
params = {"user_id": 1291248, "item_id": 17245}

resp = requests.post(events_store_url + "/put", headers=headers, params=params)

# И снова получим онлайн-рекомендации для пользователя 1291248:
params = {"user_id": 1291248, 'k': 3}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)
# Должно быть {'recs': [480204, 51496, 2623]}

In [ ]:
# Задание 5 из 6
# Протестируем метод для пользователя 1291248, сгенерировав для него несколько событий и получив пять онлайн-рекомендаций.
user_id = 1291248
event_item_ids = [41899, 102868, 5472, 5907]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})
                         
params = {"user_id": user_id, 'k': 5}

resp = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
online_recs = resp.json()
    
print(online_recs)
# Должно быть {'recs': [608474, 3590, 8921, 194373, 736131]}

In [ ]:
# Задание 6 из 6
# На примере пользователя 1291250 протестируем доработанный сервис.

# Сначала сгенерируем онлайн-события:
user_id = 1291250
event_item_ids =  [7144, 16299, 5907, 18135]

for event_item_id in event_item_ids:
    resp = requests.post(events_store_url + "/put", 
                         headers=headers, 
                         params={"user_id": user_id, "item_id": event_item_id})
    
# Получим 10 рекомендаций каждого типа для данного пользователя:
params = {"user_id": 1291250, 'k': 10}
resp_offline = requests.post(recommendations_url + "/recommendations_offline", headers=headers, params=params)
resp_online = requests.post(recommendations_url + "/recommendations_online", headers=headers, params=params)
resp_blended = requests.post(recommendations_url + "/recommendations", headers=headers, params=params)

recs_offline = resp_offline.json()["recs"]
recs_online = resp_online.json()["recs"]
recs_blended = resp_blended.json()["recs"]

print(recs_offline)
print(recs_online)
print(recs_blended)

# Качество рекомендаций также можно оценить выборочно: посмотрев, что рекомендации книг в 
# целом адекватны, по авторам, названиям. Пример кода для этого приведён ниже.
def display_items(item_ids):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    items_selected = items.query("item_id in @item_ids")[item_columns_to_use]
    items_selected = items_selected.set_index("item_id").reindex(item_ids)
    items_selected = items_selected.reset_index()
    
    display(items_selected)
    
print("Онлайн-события")
display_items(event_item_ids)
print("Офлайн-рекомендации")
display_items(recs_offline)
print("Онлайн-рекомендации")
display_items(recs_online)
print("Рекомендации")
display_items(recs_blended)